# Example of how to use cira with sci-kit learn 

understand that this is just an example and not a good model for actual trading.

In this example we will setup cira and use Liner Regression as an model and try to make prediction on a crypto asset. 

### Cira setup

In [1]:
import cira

In [2]:
cira.auth.KEY_FILE = "../alpc_key.json"
assert cira.auth.check_keys(), "the set keys dose not work"

In [3]:
portfolio = cira.Portfolio()
crypto_exchange = cira.CryptoExchange()

### Fetching data 

In [4]:
from datetime import datetime

SYMBOL = "BTC/USD"

cry = cira.Cryptocurrency(SYMBOL)    

start = datetime(2020, 7, 1)
end = datetime(2023, 7, 1)

data = cry.historical_data_df(start_date=start, end_date=end)
data.head()

open    high     low   close   volume  \
symbol  timestamp                                                            
BTC/USD 2020-07-01 05:00:00+00:00  9129.5  9270.0  9129.5  9240.5   5.4679   
        2020-07-02 05:00:00+00:00  9208.0  9244.0  8990.0  9109.0  12.2190   
        2020-07-03 05:00:00+00:00  9106.0  9125.0  9065.5  9092.0   2.6835   
        2020-07-04 05:00:00+00:00  9087.0  9169.0  9063.0  9130.0   7.4914   
        2020-07-05 05:00:00+00:00  9094.5  9119.5  8916.0  9096.0  12.8973   

                                   trade_count         vwap  
symbol  timestamp                                            
BTC/USD 2020-07-01 05:00:00+00:00         59.0  9230.826688  
        2020-07-02 05:00:00+00:00         67.0  9075.481038  
        2020-07-03 05:00:00+00:00         41.0  9112.852543  
        2020-07-04 05:00:00+00:00         36.0  9132.065382  
        2020-07-05 05:00:00+00:00         79.0  9069.180363

### Machine learning

training the model 

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


y = data[['close']]
X = data[['open', 'high', 'low']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 880836.2174796629


using the models predictions

In [33]:
def trade_stocks(predictions, threshold=0.0):
    """ Buy if 1 sell -1 """
    positions = [1 if pred > threshold else -1 for pred in predictions]
    return positions

In [34]:
positions = trade_stocks(y_pred, threshold=20_000)
print(positions)

[-1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1]
